In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil import parser
from scipy.stats import poisson
import warnings
warnings.filterwarnings('ignore')
# Today's date
today = datetime.now().date()
unique_leagues = ['england4'] #change according to league

# Time formatting
day_abbreviations = {0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"}
given_date_parsed = parser.parse(str(today))

# Manually format the day to remove leading zeros
day = given_date_parsed.day
month = given_date_parsed.strftime('%b')

# Format the date as "Su 1 Oct" or "Tu 10 Oct" without leading zero for single-digit days
formatted_date = f"{day_abbreviations[given_date_parsed.weekday()]} {day} {month}"
print(formatted_date)


final =  pd.DataFrame()
liqa = ''
next_matches = pd.DataFrame()

for i in unique_leagues:
    URL = "https://www.soccerstats.com/results.asp?league=" + i + "&pmtype=bydate"
    page = requests.get(URL)
    liqa = i
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="btable")
    sth = results.find_all("tr", class_="odd")
    sth


    date, league, home, away, ft, ht = [], [], [], [], [],[]
    for i in sth:
        date.append(i.find_all("td", align = 'right')[0].get_text(strip=True))
        league.append(liqa.capitalize())
        home.append(i.find_all("td", align = 'right')[1].get_text(strip=True))
        away.append(i.find("td", align = "left").get_text(strip = True))
        ft.append(i.find_all("td", align = 'center')[0].get_text(strip = True))
        try:
            ht.append(i.find_all("td", align = 'center')[2].get_text(strip = True))
        except IndexError as e:
            ht.append('NA')#print("Last output before error occurred:", i.find_all("td", align = 'center'))

    data = {'Date': date, 'League': league,'Home': home, 'Away': away, 'FT': ft, 'HT': ht}

# Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

# Replace empty strings with NaN
    next_df = df[(df['Date'] == formatted_date) & (df['HT'] == '')]
    next_matches = pd.concat([next_matches, next_df], ignore_index = True)
    df.replace('', pd.NA, inplace=True)

# Drop rows with NaN values
    df_cleaned = df.dropna()

#For Half-Time Results
    hthg, htag = [], []
    for i in df_cleaned['HT']:
        if i == 'NA':
            hthg.append('NA')
            htag.append('NA')
        elif i == '+' or i == '-':
            hthg.append('NA')
            htag.append('NA')
        else:
            try:
                hthg.append(int(i[1]))
                htag.append(int(i[3]))
            except IndexError as e:
                print("Last output before error occurred:", i)



#For Full-Time Results
    hg, ag, tg = [], [], []
    for i in df_cleaned['FT']:
        if len(i) < 5 or ':' in i:
            hg.append('NA')
            ag.append('NA')
            tg.append('NA')
        else:
            try:
                hghg = int(i.split(' - ')[0])
                hg.append(hghg)
                agag = int(i.split(' - ')[1])
                ag.append(agag)
                tg.append(hghg + agag)
            except:
                print(hghg + agag)

    
    df_cleaned['FTHG'], df_cleaned['FTAG'], df_cleaned['FTTG'] = hg, ag, tg
    df_cleaned['HTHG'], df_cleaned['HTAG'] = hthg, htag
    df_cleaned['HTTG'] = df_cleaned['HTHG'] + df_cleaned['HTAG']
    
    final = pd.concat([final, df_cleaned], ignore_index=True)
    
final = final[final['HT'] != 'NA']

# Define current year
current_year = datetime.now().year

# Function to parse and assign the correct year
def parse_date(date_str):
    # Extract day and month, strip any leading/trailing whitespace
    date_part = date_str[3:].strip()
    date_obj = datetime.strptime(date_part, '%d %b')
    # Assign correct year
    full_date = date_obj.replace(year=current_year)
    
    # If the parsed date is already in the future, assign the previous year
    if full_date > datetime.now():
        full_date = full_date.replace(year=current_year - 1)
    
    return full_date

# Apply the function to parse dates
final['Date'] = final['Date'].apply(parse_date)

# Find the latest date for each league
latest_dates = final.groupby('League')['Date'].max().rename('latest_date')

# Merge with the original DataFrame
final = final.merge(latest_dates, on='League')

# Calculate the time difference in days
final['time_diff'] = (final['latest_date'] - final['Date']).dt.days
combined_df = pd.concat([final.head(), final.tail()])
combined_df

Thu 1 Jan


,Date,League,Home,Away,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,latest_date,time_diff
0,2025-08-02,England4,Accrington,Gillingham,1 - 1,(0-0),1,1,2,0,0,0,2025-12-29,149
1,2025-08-02,England4,Barnet,Fleetwood,0 - 2,(0-1),0,2,2,0,1,1,2025-12-29,149
2,2025-08-02,England4,Bristol Rovers,Harrogate,0 - 1,(0-0),0,1,1,0,0,0,2025-12-29,149
3,2025-08-02,England4,Cambridge Utd,Cheltenham,1 - 0,(0-0),1,0,1,0,0,0,2025-12-29,149
4,2025-08-02,England4,Chesterfield,Barrow,1 - 0,(1-0),1,0,1,1,0,1,2025-12-29,149
271,2025-12-29,England4,Grimsby,Shrewsbury,1 - 0,(0-0),1,0,1,0,0,0,2025-12-29,0
272,2025-12-29,England4,Milton Keynes,Notts County,1 - 1,(0-1),1,1,2,0,1,1,2025-12-29,0
273,2025-12-29,England4,Salford City,Fleetwood,0 - 0,(0-0),0,0,0,0,0,0,2025-12-29,0
274,2025-12-29,England4,Tranmere,Barrow,1 - 3,(0-3),1,3,4,0,3,3,2025-12-29,0
275,2025-12-29,England4,Walsall,Oldham,1 - 2,(0-1),1,2,3,0,1,1,2025-12-29,0


In [18]:
# ============================================================
# 1. Imports
# ============================================================

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import poisson


# ============================================================
# 2. Index mappings (teams and leagues)
# ============================================================

teams = pd.concat([final['Home'], final['Away']]).unique()
leagues = final['League'].unique()

team_to_idx = {team: i for i, team in enumerate(teams)}
league_to_idx = {lg: i for i, lg in enumerate(leagues)}

n_teams = len(teams)
n_leagues = len(leagues)


# ============================================================
# 3. Negative Log-Likelihood (FT + HT, time decay)
# ============================================================

def neg_log_likelihood(params, df, team_to_idx, league_to_idx, xi):
    """
    Parameter vector structure:

    [ home_attack (n)
      away_attack (n)
      home_defence (n)
      away_defence (n)
      league_home_adv (L)
      ft_intercept
      ht_intercept ]
    """

    idx = 0
    ha = params[idx : idx + n_teams]; idx += n_teams
    aa = params[idx : idx + n_teams]; idx += n_teams
    hd = params[idx : idx + n_teams]; idx += n_teams
    ad = params[idx : idx + n_teams]; idx += n_teams
    lha = params[idx : idx + n_leagues]; idx += n_leagues
    ft_int = params[idx]
    ht_int = params[idx + 1]

    ll = 0.0

    for _, r in df.iterrows():
        h = team_to_idx[r['Home']]
        a = team_to_idx[r['Away']]
        l = league_to_idx[r['League']]

        # ---------- Expected goals ----------
        lambda_ft_home = np.exp(
            ft_int + ha[h] - ad[a] + lha[l]
        )
        lambda_ft_away = np.exp(
            ft_int + aa[a] - hd[h]
        )

        lambda_ht_home = np.exp(
            ht_int + ha[h] - ad[a] + lha[l]
        )
        lambda_ht_away = np.exp(
            ht_int + aa[a] - hd[h]
        )

        # ---------- Time decay ----------
        w = np.exp(-xi * r['time_diff'])

        # ---------- Log-likelihood ----------
        ll += w * (
            poisson.logpmf(r['FTHG'], lambda_ft_home)
            + poisson.logpmf(r['FTAG'], lambda_ft_away)
            + poisson.logpmf(r['HTHG'], lambda_ht_home)
            + poisson.logpmf(r['HTAG'], lambda_ht_away)
        )

    return -ll


# ============================================================
# 4. Initialization
# ============================================================

init_params = np.concatenate([
    np.zeros(n_teams),    # home attack
    np.zeros(n_teams),    # away attack
    np.zeros(n_teams),    # home defence
    np.zeros(n_teams),    # away defence
    np.zeros(n_leagues),  # league home advantage
    [0.1],                # FT intercept
    [-0.5]                # HT intercept (lower scoring)
])


# ============================================================
# 5. Identifiability constraints
# ============================================================

constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x[0:n_teams])},                           # home attack
    {'type': 'eq', 'fun': lambda x: np.sum(x[n_teams:2*n_teams])},                   # away attack
    {'type': 'eq', 'fun': lambda x: np.sum(x[2*n_teams:3*n_teams])},                 # home defence
    {'type': 'eq', 'fun': lambda x: np.sum(x[3*n_teams:4*n_teams])}                  # away defence
]


# ============================================================
# 6. Model fitting
# ============================================================

xi = 0.005  # time decay parameter

result = minimize(
    neg_log_likelihood,
    init_params,
    args=(final, team_to_idx, league_to_idx, xi),
    method="SLSQP",
    constraints=constraints,
    options={"maxiter": 1500, "disp": True}
)

fitted = result.x


# ============================================================
# 7. Extract fitted parameters
# ============================================================

idx = 0
home_attack  = fitted[idx : idx + n_teams]; idx += n_teams
away_attack  = fitted[idx : idx + n_teams]; idx += n_teams
home_defence = fitted[idx : idx + n_teams]; idx += n_teams
away_defence = fitted[idx : idx + n_teams]; idx += n_teams
league_home_adv = fitted[idx : idx + n_leagues]; idx += n_leagues
ft_intercept = fitted[idx]
ht_intercept = fitted[idx + 1]

team_params = pd.DataFrame({
    "team": teams,
    "home_attack": home_attack,
    "away_attack": away_attack,
    "home_defence": home_defence,
    "away_defence": away_defence
})

league_params = pd.DataFrame({
    "league": leagues,
    "home_advantage": league_home_adv
})
print(team_params, league_params)

# ============================================================
# 8. Prediction function (FT only)
# ============================================================

def predict_match(home, away, league, max_goals=6):
    h = team_to_idx[home]
    a = team_to_idx[away]
    l = league_to_idx[league]

    lambda_home = np.exp(
        ft_intercept + home_attack[h] - away_defence[a] + league_home_adv[l]
    )
    lambda_away = np.exp(
        ft_intercept + away_attack[a] - home_defence[h]
    )

    probs = {}
    for i in range(max_goals + 1):
        for j in range(max_goals + 1):
            probs[(i, j)] = poisson.pmf(i, lambda_home) * poisson.pmf(j, lambda_away)

    return lambda_home, lambda_away, probs


Optimization terminated successfully    (Exit mode 0)
            Current function value: 897.891257298097
            Iterations: 27
            Function evaluations: 2734
            Gradient evaluations: 27
               team  home_attack  away_attack  home_defence  away_defence
0        Accrington     0.019959    -0.139764      0.589145     -0.039575
1            Barnet     0.240436    -0.093714     -0.241113      0.362392
2    Bristol Rovers    -0.377348    -0.444184     -0.310599     -0.367857
3     Cambridge Utd     0.090412    -0.289554      0.773616      0.403500
4      Chesterfield     0.405792     0.161757     -0.134204     -0.273985
5    Colchester Utd     0.293808     0.409740     -0.050478      0.075363
6           Grimsby    -0.267626     0.447147      0.224278     -0.261092
7     Milton Keynes     0.451898     0.502203      0.235463     -0.060375
8      Salford City     0.215785     0.083114     -0.030296      0.198475
9        Shrewsbury    -0.245056    -0.413727     

In [30]:
home, away = 'Swindon Town', 'Gillingham'
a, b, probs = predict_match(home=home, away=away, 
                            league=unique_leagues[0].capitalize())
home_win = sum(p for (h, a), p in probs.items() if h > a)
draw = sum(p for (h, a), p in probs.items() if h == a)
away_win = sum(p for (h, a), p in probs.items() if h < a)
over_25 = 1 - sum(p for (h, a), p in probs.items() if h + a < 2.5)
btts = 1 - sum(p for (h, a), p in probs.items() if h == 0 or a == 0)

print(f"Match: {home} vs {away}")
print(f"Home win probability: {home_win:.2%}")
print(f"Draw probability: {draw:.2%}") 
print(f"Away win probability: {away_win:.2%}")
print(f"Over 2.5 goals probability: {over_25:.2%}")
print(f"BTTS probability: {btts:.2%}")

# Convert to DataFrame and pivot to table format
df_probs = pd.DataFrame([
    {'Home': h, 'Away': a, 'Probability': p} 
    for (h, a), p in probs.items()
])

# Pivot to create a matrix view
prob_table = df_probs.pivot(index='Home', columns='Away', values='Probability')
prob_table.style.format("{:.4f}").background_gradient(cmap='YlOrRd', axis=None)

Match: Swindon Town vs Gillingham
Home win probability: 44.18%
Draw probability: 27.83%
Away win probability: 27.94%
Over 2.5 goals probability: 40.54%
BTTS probability: 45.97%


Away,0,1,2,3,4,5,6
Home,,,,,,,
0,0.0997,0.0984,0.0486,0.0160,0.0039,0.0008,0.0001
1,0.1315,0.1298,0.0640,0.0211,0.0052,0.0010,0.0002
2,0.0867,0.0855,0.0422,0.0139,0.0034,0.0007,0.0001
3,0.0381,0.0376,0.0186,0.0061,0.0015,0.0003,0.0000
4,0.0126,0.0124,0.0061,0.0020,0.0005,0.0001,0.0000
5,0.0033,0.0033,0.0016,0.0005,0.0001,0.0000,0.0000
6,0.0007,0.0007,0.0004,0.0001,0.0000,0.0000,0.0000
